# Chapter 3: Functions

---

Functions are fundamental building blocks in Python programming. This chapter explores best practices for writing clear, maintainable, and effective functions.

## Item 19: Never Unpack More Than Three Return Values

### Multiple Return Values

Python allows functions to return multiple values using tuple unpacking. However, **returning more than three values becomes error-prone and hard to read**.

### Basic Multiple Return Example

In [ ]:
# Simple example showing tuple unpacking
def get_min_max(numbers):
    """Return minimum and maximum values"""
    return min(numbers), max(numbers)

lengths = [63, 73, 72, 60, 67, 66, 71, 61, 72, 70]
minimum, maximum = get_min_max(lengths)

print(f'Min: {minimum}, Max: {maximum}')

### How Multiple Returns Work

Multiple values are returned as a **tuple**, which is then unpacked:

In [ ]:
# Unpacking works the same way as multiple assignment
first, second = 1, 2
assert first == 1
assert second == 2

# Function return is equivalent
def my_function():
    return 1, 2

first, second = my_function()
assert first == 1
assert second == 2

print("✓ Basic unpacking works correctly")

### Using Starred Expressions for Multiple Returns

In [ ]:
# Catch-all unpacking with starred expressions
def get_avg_ratio(numbers):
    """Return ratios relative to average, sorted descending"""
    average = sum(numbers) / len(numbers)
    scaled = [x / average for x in numbers]
    scaled.sort(reverse=True)
    return scaled

longest, *middle, shortest = get_avg_ratio(lengths)
print(f'Longest: {longest:>4.0%}')
print(f'Shortest: {shortest:>4.0%}')
print(f'Middle values: {len(middle)} items')

### The Problem: Too Many Return Values

When functions return many values, **two major problems** arise:

1. **Easy to swap arguments** (all numeric, hard to track)
2. **Long, unreadable unpacking statements**

In [ ]:
# BAD: Too many return values
def get_stats(numbers):
    minimum = min(numbers)
    maximum = max(numbers)
    count = len(numbers)
    average = sum(numbers) / count
    
    sorted_numbers = sorted(numbers)
    middle = count // 2
    if count % 2 == 0:
        lower = sorted_numbers[middle - 1]
        upper = sorted_numbers[middle]
        median = (lower + upper) / 2
    else:
        median = sorted_numbers[middle]
    
    return minimum, maximum, average, median, count

# CORRECT unpacking
minimum, maximum, average, median, count = get_stats(lengths)
print(f'Min: {minimum}, Max: {maximum}')
print(f'Average: {average}, Median: {median}, Count {count}')

In [ ]:
# WRONG: Easy to accidentally swap median and average!
# This is valid Python but has SWAPPED median and average
minimum, maximum, median, average, count = get_stats(lengths)
print(f'Average: {average}, Median: {median}')
print("⚠️ Values are swapped but no error was raised!")

### Readability Problems with Long Unpacking

In [ ]:
# All these formats hurt readability:

# Wrapped with backslash
minimum, maximum, average, median, count = \
    get_stats(lengths)

# Parentheses wrapping
(minimum, maximum, average,
 median, count) = get_stats(lengths)

# Awkward trailing parenthesis
(minimum, maximum, average, median, count
 ) = get_stats(lengths)

print("All work, but all are hard to read!")

### Solution: Use namedtuple or Lightweight Classes

**Best Practice**: Never unpack more than **three variables**. For more complex returns, use:
- `namedtuple`
- Lightweight class
- Dataclass

In [ ]:
from collections import namedtuple

# GOOD: Use namedtuple for multiple values
Stats = namedtuple('Stats', ['minimum', 'maximum', 'average', 'median', 'count'])

def get_stats_better(numbers):
    minimum = min(numbers)
    maximum = max(numbers)
    count = len(numbers)
    average = sum(numbers) / count
    
    sorted_numbers = sorted(numbers)
    middle = count // 2
    if count % 2 == 0:
        lower = sorted_numbers[middle - 1]
        upper = sorted_numbers[middle]
        median = (lower + upper) / 2
    else:
        median = sorted_numbers[middle]
    
    return Stats(minimum, maximum, average, median, count)

# Now accessing values is clear and self-documenting
stats = get_stats_better(lengths)
print(f'Min: {stats.minimum}, Max: {stats.maximum}')
print(f'Average: {stats.average}, Median: {stats.median}')
print(f'Count: {stats.count}')

### Key Takeaways

| ✓ Good Practice | ✗ Bad Practice |
|----------------|----------------|
| Return 1-3 values max | Return 4+ values |
| Use namedtuple for complex returns | Rely on position for many values |
| Use starred expressions for catch-all | Mix too many return types |

---

## Item 20: Prefer Raising Exceptions to Returning None

### The Problem with Returning None

Returning `None` for special cases is **error-prone** because:
- `None` evaluates to `False` in conditionals
- Zero, empty string, empty list also evaluate to `False`
- Easy to accidentally misinterpret results

In [ ]:
# BAD: Returning None for errors
def careful_divide_bad(a, b):
    try:
        return a / b
    except ZeroDivisionError:
        return None

# Checking for None
x, y = 1, 0
result = careful_divide_bad(x, y)
if result is None:
    print('Invalid inputs')
else:
    print(f'Result: {result}')

### The Trap: Zero Returns

In [ ]:
# BUG: This looks like an error check, but it's wrong!
x, y = 0, 5
result = careful_divide_bad(x, y)

# WRONG: This triggers even though the division succeeded!
if not result:
    print('Invalid inputs')  # This runs! But shouldn't
else:
    print(f'Result: {result}')

### Approach 1: Return Status Tuple (Not Recommended)

In [ ]:
# MEDIOCRE: Return success status + result
def careful_divide_tuple(a, b):
    try:
        return True, a / b
    except ZeroDivisionError:
        return False, None

# Forces caller to check status
success, result = careful_divide_tuple(x, y)
if not success:
    print('Invalid inputs')
else:
    print(f'Result: {result}')

In [ ]:
# PROBLEM: Callers can ignore the status using underscore
_, result = careful_divide_tuple(x, y)
if not result:
    print('Invalid inputs')  # Still runs incorrectly!

### Approach 2: Raise Exceptions (BEST PRACTICE)

In [ ]:
# GOOD: Raise exceptions for special cases
def careful_divide(a, b):
    try:
        return a / b
    except ZeroDivisionError as e:
        raise ValueError('Invalid inputs')

# Caller handles exception explicitly
x, y = 5, 2
try:
    result = careful_divide(x, y)
except ValueError:
    print('Invalid inputs')
else:
    print(f'Result is {result:.1f}')

In [ ]:
# Exception is raised for actual errors
try:
    result = careful_divide(5, 0)
except ValueError as e:
    print(f'Caught error: {e}')

### Using Type Annotations with Exceptions

In [ ]:
# BEST: Type annotations + docstring documenting exceptions
def careful_divide(a: float, b: float) -> float:
    """Divides a by b.
    
    Raises:
        ValueError: When the inputs cannot be divided.
    """
    try:
        return a / b
    except ZeroDivisionError as e:
        raise ValueError('Invalid inputs')

# Type annotations make it clear the function always returns float
# Never returns None
result = careful_divide(10, 2)
print(f'Result: {result}')

### Comparison: None vs Exceptions

| Approach | Pros | Cons |
|----------|------|------|
| **Return None** | Simple | Error-prone, confuses falsy values |
| **Return (status, value)** | Explicit status | Can be ignored, verbose |
| **Raise Exception** ✓ | Clear, forces handling | Slightly more verbose |

---

## Item 21: Know How Closures Interact with Variable Scope

### Understanding Closures

A **closure** is a function that refers to variables from the scope in which it was defined.

In [ ]:
# Basic closure example
def sort_priority(values, group):
    def helper(x):
        if x in group:
            return (0, x)  # High priority
        return (1, x)      # Low priority
    values.sort(key=helper)

numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {2, 3, 5, 7}
sort_priority(numbers, group)
print(numbers)

### Why Closures Work

Three key Python features enable closures:

1. **Closures** can reference variables from enclosing scopes
2. **Functions are first-class objects** (can be passed around)
3. **Tuple comparison** works element-by-element

In [ ]:
# Demonstrating first-class functions
def make_multiplier(n):
    def multiply(x):
        return x * n  # 'n' comes from enclosing scope
    return multiply

times_three = make_multiplier(3)
print(f'3 × 5 = {times_three(5)}')
print(f'3 × 10 = {times_three(10)}')

### The Scoping Problem

Attempting to modify enclosing scope variables creates a **scoping bug**:

In [ ]:
# BUG: Trying to modify enclosing scope
def sort_priority2(numbers, group):
    found = False  # Outer scope
    def helper(x):
        if x in group:
            found = True  # BUG: Creates NEW local variable!
            return (0, x)
        return (1, x)
    numbers.sort(key=helper)
    return found

numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {2, 3, 5, 7}
found = sort_priority2(numbers, group)
print('Found:', found)  # False - WRONG!
print(numbers)          # Sorted correctly though

### Variable Scope Resolution Order

Python resolves variable references in this order:

1. Current function's scope
2. Enclosing scopes
3. Global scope
4. Built-in scope

**Assignment** creates a local variable unless told otherwise!

In [ ]:
# Demonstrating scope resolution
try:
    foo = does_not_exist * 5
except NameError as e:
    print(f"NameError: {e}")

### Solution 1: Using `nonlocal`

In [ ]:
# GOOD: Use nonlocal to modify enclosing scope
def sort_priority3(numbers, group):
    found = False
    def helper(x):
        nonlocal found  # Tells Python to look in enclosing scope
        if x in group:
            found = True
            return (0, x)
        return (1, x)
    numbers.sort(key=helper)
    return found

numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {2, 3, 5, 7}
found = sort_priority3(numbers, group)
print('Found:', found)  # True - CORRECT!
print(numbers)

### Solution 2: Helper Class (For Complex State)

In [ ]:
# BETTER: Use a class for more complex state
class Sorter:
    def __init__(self, group):
        self.group = group
        self.found = False
    
    def __call__(self, x):
        if x in self.group:
            self.found = True
            return (0, x)
        return (1, x)

numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {2, 3, 5, 7}
sorter = Sorter(group)
numbers.sort(key=sorter)

assert sorter.found is True
print('Found:', sorter.found)
print(numbers)

### When to Use Each Approach

| Approach | Best For |
|----------|----------|
| **Simple closures** | Reading enclosing variables |
| **nonlocal** | Simple state modification |
| **Helper class** ✓ | Complex state or multiple values |

---

## Item 22: Reduce Visual Noise with Variable Positional Arguments

### The Power of *args

Variable positional arguments (`*args`) make function calls cleaner by accepting any number of positional arguments.

In [ ]:
# WITHOUT *args - verbose and noisy
def log_old(message, values):
    if not values:
        print(message)
    else:
        values_str = ', '.join(str(x) for x in values)
        print(f'{message}: {values_str}')

log_old('My numbers are', [1, 2])
log_old('Hi there', [])  # Annoying empty list!

In [ ]:
# WITH *args - clean and elegant
def log(message, *values):
    if not values:
        print(message)
    else:
        values_str = ', '.join(str(x) for x in values)
        print(f'{message}: {values_str}')

log('My numbers are', 1, 2)
log('Hi there')  # Much better!

### Using * Operator to Unpack Sequences

In [ ]:
# Unpack a list as positional arguments
favorites = [7, 33, 99]
log('Favorite colors', *favorites)

# Without *, would pass the list itself as single argument
log('Favorite colors', favorites)  # Different behavior

### Problem 1: Memory Consumption with Generators

In [ ]:
# WARNING: *args exhausts generators into memory
def my_generator():
    for i in range(10):
        yield i

def my_func(*args):
    print(args)

it = my_generator()
my_func(*it)  # Entire generator is consumed into tuple!

In [ ]:
# For large generators, this could cause memory problems
def large_generator():
    for i in range(1000000):
        yield i

# DON'T DO THIS with huge generators!
# my_func(*large_generator())  # Would consume ~8MB of memory

### Problem 2: Adding New Positional Arguments

In [ ]:
# Initial function
def log_v1(message, *values):
    if not values:
        print(f'{message}')
    else:
        values_str = ', '.join(str(x) for x in values)
        print(f'{message}: {values_str}')

log_v1('Favorites', 7, 33)
log_v1('Hi there')

In [ ]:
# Adding a new positional argument BREAKS old callers!
def log_v2(sequence, message, *values):
    if not values:
        print(f'{sequence} - {message}')
    else:
        values_str = ', '.join(str(x) for x in values)
        print(f'{sequence} - {message}: {values_str}')

log_v2(1, 'Favorites', 7, 33)  # New with sequence - OK
log_v2(1, 'Hi there')           # New message only - OK
log_v2('Favorite numbers', 7, 33)  # Old usage - BREAKS!
# Now 'Favorite numbers' is used as sequence!

### Best Practices for *args

✓ **Use *args when:**
- Number of arguments is small and known
- Passing literals or variable names
- Improving readability

✗ **Avoid *args when:**
- Working with large generators
- Function interface might expand
- Arguments could be confused

**Solution**: Use keyword-only arguments for extensibility (see Item 25)

---

## Item 23: Provide Optional Behavior with Keyword Arguments

### Keyword Arguments Basics

In [ ]:
# Function with positional parameters
def remainder(number, divisor):
    return number % divisor

# All these calls are equivalent:
assert remainder(20, 7) == 6
assert remainder(20, divisor=7) == 6
assert remainder(number=20, divisor=7) == 6
assert remainder(divisor=7, number=20) == 6

print("✓ All calling styles produce same result")

In [ ]:
# Positional arguments must come before keyword arguments
try:
    remainder(number=20, 7)
except SyntaxError:
    print("SyntaxError: positional argument follows keyword argument")

In [ ]:
# Each argument can only be specified once
try:
    remainder(20, number=7)
except TypeError as e:
    print(f"TypeError: {e}")

### Using ** Operator with Dictionaries

In [ ]:
# Pass dictionary contents as keyword arguments
my_kwargs = {
    'number': 20,
    'divisor': 7,
}
assert remainder(**my_kwargs) == 6

# Mix with positional or keyword arguments
my_kwargs = {'divisor': 7}
assert remainder(number=20, **my_kwargs) == 6

# Multiple ** operators
my_kwargs = {'number': 20}
other_kwargs = {'divisor': 7}
assert remainder(**my_kwargs, **other_kwargs) == 6

### Catching All Keyword Arguments with **kwargs

In [ ]:
# Accept any keyword arguments
def print_parameters(**kwargs):
    for key, value in kwargs.items():
        print(f'{key} = {value}')

print_parameters(alpha=1.5, beta=9, gamma=4)

### Benefit 1: Clarity

In [ ]:
# Positional - unclear which is number, which is divisor
result = remainder(20, 7)

# Keyword - immediately obvious
result = remainder(number=20, divisor=7)
print(f"Result: {result}")

### Benefit 2: Default Values

In [ ]:
# Function for calculating flow rate
def flow_rate(weight_diff, time_diff, period=1):
    return (weight_diff / time_diff) * period

weight_diff = 0.5
time_diff = 3

# Default: per second
flow_per_second = flow_rate(weight_diff, time_diff)
print(f'{flow_per_second:.3} kg per second')

# Specify: per hour
flow_per_hour = flow_rate(weight_diff, time_diff, period=3600)
print(f'{flow_per_hour:.3} kg per hour')

### Benefit 3: Backward Compatibility

In [ ]:
# Extend function without breaking existing code
def flow_rate_extended(weight_diff, time_diff,
                       period=1, units_per_kg=1):
    return ((weight_diff * units_per_kg) / time_diff) * period

# Old callers still work
flow_per_second = flow_rate_extended(weight_diff, time_diff)
print(f'{flow_per_second:.3} kg per second')

# New callers can use new feature
pounds_per_hour = flow_rate_extended(
    weight_diff, time_diff,
    period=3600, units_per_kg=2.2
)
print(f'{pounds_per_hour:.3} pounds per hour')

### Warning: Positional Optional Arguments Can Be Confusing

In [ ]:
# CONFUSING: What do 3600 and 2.2 represent?
pounds_per_hour = flow_rate_extended(weight_diff, time_diff, 3600, 2.2)

# CLEAR: Use keywords for optional arguments
pounds_per_hour = flow_rate_extended(
    weight_diff, time_diff,
    period=3600, units_per_kg=2.2
)
print(f'{pounds_per_hour:.3} pounds per hour')

### Best Practices Summary

| Practice | Benefit |
|----------|----------|
| Use keyword arguments | Clarity at call site |
| Provide default values | Reduce repetition |
| Add optional keywords | Backward compatibility |
| Always use keywords for optional args | Avoid confusion |

---

## Item 24: Use None and Docstrings to Specify Dynamic Default Arguments

### The Problem: Mutable Default Arguments

In [ ]:
from datetime import datetime
from time import sleep

# BUG: Default argument evaluated only once at function definition!
def log_bad(message, when=datetime.now()):
    print(f'{when}: {message}')

log_bad('Hi there!')
sleep(0.1)
log_bad('Hello again!')  # Same timestamp!

### Why This Happens

Default argument values are evaluated **only once**, when the function is defined (at module load time).

After the module loads, `datetime.now()` is **never called again**!

### Solution: Use None with Docstring

In [ ]:
# GOOD: Use None as default, allocate in function body
def log(message, when=None):
    """Log a message with a timestamp.
    
    Args:
        message: Message to print.
        when: datetime of when the message occurred.
            Defaults to the present time.
    """
    if when is None:
        when = datetime.now()
    print(f'{when}: {message}')

log('Hi there!')
sleep(0.1)
log('Hello again!')  # Different timestamps now!

### The Mutable Default Trap

In [ ]:
import json

# DANGER: Mutable default argument
def decode_bad(data, default={}):
    try:
        return json.loads(data)
    except ValueError:
        return default

# All calls share the SAME dictionary!
foo = decode_bad('bad data')
foo['stuff'] = 5

bar = decode_bad('also bad')
bar['meep'] = 1

print('Foo:', foo)
print('Bar:', bar)
print('They are the same object!', foo is bar)

### Correct Implementation

In [ ]:
# GOOD: Use None, create new dict in function
def decode(data, default=None):
    """Load JSON data from a string.
    
    Args:
        data: JSON data to decode.
        default: Value to return if decoding fails.
            Defaults to an empty dictionary.
    """
    try:
        return json.loads(data)
    except ValueError:
        if default is None:
            default = {}
        return default

# Now each call gets its own dictionary
foo = decode('bad data')
foo['stuff'] = 5

bar = decode('also bad')
bar['meep'] = 1

print('Foo:', foo)
print('Bar:', bar)
assert foo is not bar

### Using Type Annotations with None Defaults

In [ ]:
from typing import Optional

# Type annotations make None default explicit
def log_typed(message: str,
               when: Optional[datetime] = None) -> None:
    """Log a message with a timestamp.
    
    Args:
        message: Message to print.
        when: datetime of when the message occurred.
            Defaults to the present time.
    """
    if when is None:
        when = datetime.now()
    print(f'{when}: {message}')

log_typed('Type-annotated message')

### Common Dynamic Default Scenarios

| Default Value Type | Problem | Solution |
|-------------------|---------|----------|
| `datetime.now()` | Evaluated once at import | Use `None`, call in function |
| `{}` (empty dict) | Shared across calls | Use `None`, create `{}` in function |
| `[]` (empty list) | Shared across calls | Use `None`, create `[]` in function |
| Any mutable object | Shared state | Use `None`, create fresh object |

---

## Item 25: Enforce Clarity with Keyword-Only and Positional-Only Arguments

### The Problem: Ambiguous Arguments

In [ ]:
# Confusing Boolean arguments
def safe_division_bad(number, divisor,
                      ignore_overflow,
                      ignore_zero_division):
    try:
        return number / divisor
    except OverflowError:
        if ignore_overflow:
            return 0
        else:
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise

# CONFUSING: Which Boolean is which?
result = safe_division_bad(1.0, 10**500, True, False)
print(result)

result = safe_division_bad(1.0, 0, False, True)
print(result)

### Improvement: Optional Keyword Arguments

In [ ]:
# Better: Default values
def safe_division_b(number, divisor,
                    ignore_overflow=False,
                    ignore_zero_division=False):
    try:
        return number / divisor
    except OverflowError:
        if ignore_overflow:
            return 0
        else:
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise

# Much clearer with keywords
result = safe_division_b(1.0, 10**500, ignore_overflow=True)
print(result)

result = safe_division_b(1.0, 0, ignore_zero_division=True)
print(result)

In [ ]:
# Problem: Still can use positional arguments (confusing!)
result = safe_division_b(1.0, 10**500, True, False)
assert result == 0

### Solution: Keyword-Only Arguments (*)

In [ ]:
# BEST: Force keyword-only arguments with *
def safe_division_c(number, divisor, *,
                    ignore_overflow=False,
                    ignore_zero_division=False):
    try:
        return number / divisor
    except OverflowError:
        if ignore_overflow:
            return 0
        else:
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise

# Now positional arguments fail
try:
    safe_division_c(1.0, 10**500, True, False)
except TypeError as e:
    print(f"TypeError: {e}")

In [ ]:
# Keywords work perfectly
result = safe_division_c(1.0, 0, ignore_zero_division=True)
assert result == float('inf')
print("✓ Keyword-only arguments enforce clarity")

### Problem: Parameter Names as Interface

In [ ]:
# Callers can use parameter names
assert safe_division_c(number=2, divisor=5) == 0.4
assert safe_division_c(divisor=5, number=2) == 0.4
assert safe_division_c(2, divisor=5) == 0.4

In [ ]:
# Problem: Renaming parameters breaks code!
def safe_division_c_renamed(numerator, denominator, *,
                            ignore_overflow=False,
                            ignore_zero_division=False):
    try:
        return numerator / denominator
    except OverflowError:
        if ignore_overflow:
            return 0
        else:
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise

# Old code breaks!
try:
    safe_division_c_renamed(number=2, divisor=5)
except TypeError as e:
    print(f"TypeError: {e}")

### Solution: Positional-Only Arguments (/)

Python 3.8+ supports positional-only parameters using `/`

In [ ]:
# BEST: Positional-only parameters before /
def safe_division_d(numerator, denominator, /, *,
                    ignore_overflow=False,
                    ignore_zero_division=False):
    try:
        return numerator / denominator
    except OverflowError:
        if ignore_overflow:
            return 0
        else:
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise

# Positional arguments work
assert safe_division_d(2, 5) == 0.4
print("✓ Positional arguments work")

In [ ]:
# Keywords fail for positional-only parameters
try:
    safe_division_d(numerator=2, denominator=5)
except TypeError as e:
    print(f"TypeError: {e}")
    print("✓ Cannot use keywords for positional-only params")

### Flexible Parameters: Between / and *

In [ ]:
# Parameters between / and * can be passed either way
def safe_division_e(numerator, denominator, /,
                    ndigits=10, *,
                    ignore_overflow=False,
                    ignore_zero_division=False):
    try:
        fraction = numerator / denominator
        return round(fraction, ndigits)
    except OverflowError:
        if ignore_overflow:
            return 0
        else:
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise

# All these work!
result = safe_division_e(22, 7)
print(result)

result = safe_division_e(22, 7, 5)  # ndigits by position
print(result)

result = safe_division_e(22, 7, ndigits=2)  # ndigits by keyword
print(result)

### Parameter Type Summary

```python
def func(pos_only, /, pos_or_kwd, *, kwd_only):
    pass
```

| Position | Type | Symbol | Usage |
|----------|------|--------|-------|
| Before `/` | Positional-only | `/` | Must use position |
| Between `/` and `*` | Flexible | — | Position or keyword |
| After `*` | Keyword-only | `*` | Must use keyword |

---

## Item 26: Define Function Decorators with functools.wraps

### What Are Decorators?

Decorators allow you to **wrap functions** to modify their behavior before and after execution.

In [ ]:
# Simple decorator to trace function calls
def trace(func):
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        print(f'{func.__name__}({args!r}, {kwargs!r}) '
              f'-> {result!r}')
        return result
    return wrapper

# Apply decorator using @ syntax
@trace
def fibonacci(n):
    """Return the n-th Fibonacci number"""
    if n in (0, 1):
        return n
    return (fibonacci(n - 2) + fibonacci(n - 1))

# Test the decorated function
result = fibonacci(4)
print(f'\nFinal result: {result}')

### How @ Syntax Works

In [ ]:
# Using @ is equivalent to:
def fibonacci_manual(n):
    """Return the n-th Fibonacci number"""
    if n in (0, 1):
        return n
    return (fibonacci_manual(n - 2) + fibonacci_manual(n - 1))

fibonacci_manual = trace(fibonacci_manual)

print("Manual decoration:")
fibonacci_manual(3)

### The Problem: Lost Metadata

In [ ]:
# Decorated function loses its identity
print(f"Function name: {fibonacci}")
print(f"Function __name__: {fibonacci.__name__}")

In [ ]:
# help() shows wrapper, not fibonacci!
help(fibonacci)

In [ ]:
# Pickle fails
import pickle

try:
    pickle.dumps(fibonacci)
except AttributeError as e:
    print(f"AttributeError: {e}")

### Solution: functools.wraps

In [ ]:
from functools import wraps

# GOOD: Use @wraps to preserve metadata
def trace_better(func):
    @wraps(func)  # This is the key!
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        print(f'{func.__name__}({args!r}, {kwargs!r}) '
              f'-> {result!r}')
        return result
    return wrapper

@trace_better
def fibonacci_fixed(n):
    """Return the n-th Fibonacci number"""
    if n in (0, 1):
        return n
    return (fibonacci_fixed(n - 2) + fibonacci_fixed(n - 1))

# Now metadata is preserved!
print(f"Function __name__: {fibonacci_fixed.__name__}")

In [ ]:
# help() now shows the correct docstring
help(fibonacci_fixed)

In [ ]:
# Pickle works now
pickled = pickle.dumps(fibonacci_fixed)
print(f"✓ Successfully pickled: {len(pickled)} bytes")

### What @wraps Preserves

`@wraps` copies these attributes from the wrapped function:
- `__name__` - Function name
- `__module__` - Module name
- `__doc__` - Docstring
- `__annotations__` - Type annotations
- `__qualname__` - Qualified name
- `__dict__` - Instance dictionary

In [ ]:
# Demonstrate preserved attributes
print(f"Name: {fibonacci_fixed.__name__}")
print(f"Module: {fibonacci_fixed.__module__}")
print(f"Doc: {fibonacci_fixed.__doc__}")
print(f"Qualname: {fibonacci_fixed.__qualname__}")

### Real-World Decorator Example

In [ ]:
import time

def timing_decorator(func):
    """Decorator that measures function execution time"""
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f'{func.__name__} took {end - start:.4f} seconds')
        return result
    return wrapper

@timing_decorator
def slow_function(n):
    """Simulate slow computation"""
    time.sleep(0.1)
    return sum(range(n))

result = slow_function(1000)
print(f'Result: {result}')

### Decorator Best Practices

✓ **Always use `@wraps`** when defining decorators

✓ **Use `*args` and `**kwargs`** to accept any arguments

✓ **Return the result** from the wrapped function

✓ **Document behavior** in decorator docstring

---

## Chapter 3 Summary

### Key Takeaways

| Item | Key Principle |
|------|---------------|
| **19** | Never unpack more than 3 return values; use namedtuple or classes |
| **20** | Raise exceptions instead of returning None for errors |
| **21** | Use `nonlocal` or classes for closures that modify enclosing scope |
| **22** | Use `*args` for optional positional arguments (with caution) |
| **23** | Provide optional behavior with keyword arguments |
| **24** | Use `None` and docstrings for dynamic default arguments |
| **25** | Use `/` for positional-only and `*` for keyword-only arguments |
| **26** | Always use `functools.wraps` when writing decorators |

### Best Practices Checklist

- ✓ Return simple tuples (≤3 values) or use namedtuple
- ✓ Raise exceptions for special cases
- ✓ Use type annotations for clarity
- ✓ Leverage keyword arguments for optional behavior
- ✓ Avoid mutable default arguments
- ✓ Enforce argument passing style with `/` and `*`
- ✓ Always use `@wraps` in decorators